In [15]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf

env = gym.make("FrozenLake-v0")

[2017-01-26 15:49:17,355] Making new env: FrozenLake-v0


In [34]:
def next_action(actions):
    n = len(actions)
    return choice(n, p=actions)
    
def select_columns(tensor, indexes):
    idx = tf.stack((tf.range(tf.shape(indexes)[0]), indexes), 1)
    return tf.gather_nd(tensor, idx)

def discount(rewards, y):
    r_accum = 0.0
    gains = []
    for r in reversed(list(rewards)):
        r_accum = r + y * r_accum 
        gains.insert(0, r_accum)
        
    return gains

In [35]:
n_actions = env.action_space.n
n_states = env.observation_space.n
model_name = "deep-q.model"
model_path = "/models/" + model_name
y = 0.95

graph = tf.Graph()
with graph.as_default():
    with tf.device("cpu:0"):
        S = tf.placeholder(tf.int32, [None], name='s')
        A = tf.placeholder(tf.int32, [None], name='a')
        R = tf.placeholder(tf.float32, [None], name='r')
        LR = tf.placeholder(tf.float32, [], name='lr')

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        PS = Pipe(
            S,
            T.one_hot(n_states)
            .softmax_layer(4, **ops)
        )
        
        PSA = select_columns(PS, A)
        
        Loss = -tf.reduce_sum(tf.log(PSA) * R)
        Update = tf.train.GradientDescentOptimizer(LR).minimize(Loss)

        Writer = tf.summary.FileWriter('/logs/' +  model_name)
        Saver = tf.train.Saver()

In [36]:
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    with tf.device("cpu:0"):
        sess.run(tf.global_variables_initializer())
    #     saver.restore(sess, model_path)

        r_total = 0
        k = 2000.0
        
        buffer = []

        for t in range(200000):
            lr = 0.5 * k / (k + t)

            s = env.reset()
            
            ep_buffer = []
            
            
            ss =  []
            as_ = []
            rs =  []

            while True:
                #get policy
                ps = sess.run(PS, feed_dict={S: [s]})[0]
                
                #next action
                a = next_action(ps)

                #take step
                s1, r, done, info = env.step(a)
                r_total += r
                
                #update buffer
                ep_buffer.append((s, a, r, s1))
                
                #save variables
                ss.append(s), rs.append(r), as_.append(a)

                #update state
                s = s1

                if done:
                    break
            
            gs = discount(rs, y)
            
            #train
            sess.run(Update, feed_dict={
                S: ss, A: as_, R: gs, 
                LR: lr
            })

            if t % 500 == 0:
                print r_total, "of", 500, ", lr:", lr
                r_total = 0
                Saver.save(sess, model_path)


0.0 of 500 , lr: 0.5
5.0 of 500 , lr: 0.4
24.0 of 500 , lr: 0.333333333333
44.0 of 500 , lr: 0.285714285714
86.0 of 500 , lr: 0.25
144.0 of 500 , lr: 0.222222222222
181.0 of 500 , lr: 0.2
193.0 of 500 , lr: 0.181818181818


KeyboardInterrupt: 

In [ ]:
s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    Saver.restore(sess, model_path)
    
    for i in range(100):
        qs = sess.run(QS, feed_dict={S: [s]})
        a = next_action(qs, 0)
        s, r, done, info = env.step(a)
        env.render()
        print("")

        if done:
            print(r)
            break